In [1]:
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')

import pandas as pd

In [2]:
ETF_Summary = pd.read_csv('ETF_summary_df.csv') # Loading ETF summary data from the input csv file.
ETF_Summary = ETF_Summary[ETF_Summary.Regressor_Slope > 0] # Removing ETFs with negative regressor slope which indicate that these ETFs are following a negative trend in terms of their value.

In [3]:
def ETF_Recommender_Reset(User_ID, ETF_Summary):
    # This function calls back ETF_Recommender function if input parameters do not match the output criteria.
    # User_ID: Unique user ID - Taken directly from the ETF_Recommender function.
    # ETF_Summary: ETF summary data frame - Taken directly from the ETF_Recommender function.
    
    # Reconfiguration message
    clear_output(wait=False)
    print("Required number of ETFs not found for the selected portfolio size!")
    print("Please select different constraints. At any time, please enter \"exit\" to quit.")
    
    exit = False # Arguement to exit the recommendation process.
    new_args = [] # Creating an empty list to store new input paramters.
    for i in range(3):
        if i == 0: # Collecting user class information.
            input_i = input("Please enter new ETF Class preferences: ") # Requesting user class input.
            input_i = input_i.split(',') # Converting user class information to required list format.
            if input_i[0].lower() == 'exit': # checking for exit command.
                exit = True # Setting the boolean to True if the user chose to exit.
                break # exiting the function.
        elif i == 1:
            input_i = input("Please update Risk Class preference: ") # Requesting risk class input.
            if input_i.lower() == 'exit': # checking for exit command.
                exit = True # Setting the boolean to True if the user chose to exit.
                break # exiting the function.
        else:
            input_i = input("Please update Portfolio_Size: ") # Requesting portfolio size.
            if input_i.lower() == 'exit': # checking for exit command.
                exit = True # Setting the boolean to True if the user chose to exit.
                break # exiting the function.
            input_i = int(input_i)
        
        new_args.append(input_i) # updating the list with the current input.      
    if exit:
        return exit, _ # If user chose to exit, an exit command is sent to the main ETF_Recommender function.
    else:
        return exit, ETF_Recommender(User_ID, ETF_Summary,*new_args) # re-running the original ETF_Recommender function with the new inputs.

In [4]:
def ETF_Recommender(User_ID, ETF_Summary, ETF_Class, Risk_Class, Portfolio_Size=5):
    # Outputs ETF option for a give selection of ETF classes. 
    # The function eliminates all the ETF optios with respective risk classes greater than the chosen risk class.
    # Function attempts to output selected number of ETF options with maximum historical returns. 
    #--------> If unsuccessful, a new combination of inputs is requested.
    # User_ID: Unique user ID
    # ETF_Class: User ETF category preference. 
    #             Use ['None'] if no specific preference.
    #             Multiple class preferences can be enterd in a list format.
    # ETF_Summary: ETF summary data frame.
    # Risk_Class: User risk category in a string format.
    # Portfolio_Size: Number of ETFs to use in a portfolio. Default is 5.
    
    if ETF_Class[0] == 'None': # Checking if a user has any ETF class preference.
        Class_Selection = False
    else:
        Class_Selection = True
    
    if Class_Selection: # If the user has any class preference...
        # Filtering the ETF_Summary data frame by class preference and user risk class and extracting ETFs based on required portfolio size.
        ETF_Summary_filtered = ETF_Summary[(ETF_Summary['Class'].isin(ETF_Class)) & (ETF_Summary['Risk_Category'] <= Risk_Class)].reset_index(
        ).drop(columns=['index','Class','Risk_Category']).sort_values(by='Regressor_Slope').head(Portfolio_Size)
    else: # If the user has no class preference...
        # Filtering the ETF_Summary data frame by user risk class and extracting ETFs based on required portfolio size.
        ETF_Summary_filtered = ETF_Summary[ETF_Summary['Risk_Category'] <= Risk_Class].reset_index(
        ).drop(columns=['index','Class','Risk_Category']).sort_values(by='Regressor_Slope').head(Portfolio_Size)

    if len(ETF_Summary_filtered) < Portfolio_Size: # Checking if required number of ETFs the satisfy user preferences are not available.
        exit, ETF_Recommendation = ETF_Recommender_Reset(User_ID, ETF_Summary) # Calling ETF_Recommender_Reset function to request new inputs to re-run the function.
        if exit: # Exiting the function if the user chose to exit the re-configuration process.
            return
        else:
            ETF_Recommendation.to_csv('ETF_Recommendation_' + User_ID +'.csv', index=False) # Writing ETF selection to a csv file.
        return ETF_Recommendation
    else:
        ETF_Recommendation = ETF_Summary_filtered
        ETF_Recommendation.to_csv('ETF_Recommendation_' + User_ID +'.csv', index=False) # Writing ETF selection to a csv file.
        return ETF_Recommendation # Returns recommended ETFs for observability.

In [5]:
ETF_Recommender('User_750', ETF_Summary, ['None'], 'D', 5)

,Tickers,Mean_Variance,Regressor_Slope,Sharpe
135,REZ,0.902853,0.000361,-0.058812
94,PDBC,1.069663,0.000380,-0.378527
6,INDS,0.956495,0.000599,-0.011596
1,DBC,1.042336,0.000631,-0.399053
84,EWM,0.991706,0.002059,-0.038853


In [6]:
ETF_Summary

,Tickers,Mean_Variance,Regressor_Slope,Sharpe,Class,Risk_Category
0,EWC,0.945912,0.011811,0.576545,D,C
1,DBC,1.042336,0.000631,-0.399053,B,D
2,FNDX,0.885791,0.040665,1.269875,A,C
3,DDWM,0.868818,0.010440,0.659815,E,B
4,RWK,0.916186,0.073757,0.763167,E,C
...,...,...,...,...,...,...
169,DSTL,0.869681,0.036566,1.447580,D,B
170,JPME,0.871130,0.036201,0.572180,B,B
171,FLJH,0.950982,0.032639,2.037253,E,C
172,FFIU,0.982763,0.002286,0.081048,D,C
